In [1]:
from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time
import copy 
import multiprocess as mp

import gym
from env import FrozenLakeCustom, FrozenLakeSimulator

from mcts_haver import run_mcts_trial
from value_iteration import value_iteration

from config import parse_args
from utils import MultiProcess

import logging
logger = logging.getLogger()
logger.setLevel(logging.FATAL)

In [2]:
np.random.seed(0)
random.seed(0)

# params
args = parse_args()

#
env_id = "FrozenLake-v1"
env = FrozenLakeCustom(
    map_name=args["map_name"], is_slippery=args["is_slippery"],
    render_mode=args["render_mode"])

simulator = FrozenLakeSimulator(env.P)

V_vit, Q_vit = value_iteration(
    simulator, args["gamma"], args["vit_thres"])
# global Q_vit_g = Q_vit
        
for state in range(simulator.num_states):
    logging.warning(f"\n-> state = {state}")
    logging.warning(f"V[state] = {V_vit[state]:0.4f}")
    for action in range(simulator.num_actions):
        logging.warning(f"Q[state][action] = {Q_vit[state][action]:0.4f}")
    logging.warning(f"best_action={np.argmax(Q_vit[state])}")
    
manager = mp.Manager()
ep_reward_list = manager.list()
Q_mcts_list = manager.list()

def run_trial(i_trial, Q_vit, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

#     env = FrozenLakeCustom(
#         map_name=args["map_name"], is_slippery=args["is_slippery"],
#         render_mode=args["render_mode"])

#     simulator = FrozenLakeSimulator(env.P)

    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, args)

    ep_reward_list.append(ep_reward)
    Q_mcts_list.append(Q_mcts)
    return ep_reward

In [3]:
args["update_method"] = "avg"
args["rollout_method"] = ""

print(f"num_trials = {args['num_trials']}")
print(f"mcts_num_trajectories = {args['mcts_num_trajectories']}")

Q_mcts_dict = defaultdict()

hparam_ucb_scale_ary = np.arange(10, 100, 10)
hparam_ucb_scale_ary = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_ucb_scale_ary = [2**i for i in range(8, 12)]

best_param = None
max_reward_mean = -np.inf

for hparam_ucb_scale in hparam_ucb_scale_ary:
    start_time = time.time()
    
    print(f"\n-> hparam_ucb_scale = {hparam_ucb_scale}")
    args["hparam_ucb_scale"] = hparam_ucb_scale
    
    processes = MultiProcess()
    processes.run(
        run_trial, [(i, Q_vit, args) for i in range(args["num_trials"])])
    
    reward_mean = np.mean(ep_reward_list)
    reward_std = np.std(ep_reward_list, ddof=1) if len(ep_reward_list) > 1 else 0
    print(f"reward = {reward_mean:0.2f} +/- {reward_std:0.2f}")
    
    Q_mcts_dict[f"{hparam_ucb_scale}"] = copy.deepcopy(Q_mcts_list)
    
    if reward_mean > max_reward_mean:
        max_reward_mean = reward_mean 
        best_param = hparam_ucb_scale
        
    ep_reward_list[:] = []
    Q_mcts_list[:] = []
    
    end_time = time.time()
    print(f"it takes {end_time-start_time:0.4f}")

num_trials = 20
mcts_num_trajectories = 2000

-> hparam_ucb_scale = 1
reward = -101.00 +/- 0.00
it takes 0.9820

-> hparam_ucb_scale = 2
reward = -101.00 +/- 0.00
it takes 0.9640

-> hparam_ucb_scale = 4
reward = -101.00 +/- 0.00
it takes 1.0752

-> hparam_ucb_scale = 8
reward = -101.00 +/- 0.00
it takes 1.0538

-> hparam_ucb_scale = 16
reward = -101.00 +/- 0.00
it takes 1.2567

-> hparam_ucb_scale = 32
reward = -4.00 +/- 0.00
it takes 9.0315

-> hparam_ucb_scale = 64
reward = -4.00 +/- 0.00
it takes 9.0358

-> hparam_ucb_scale = 128
reward = -4.00 +/- 0.00
it takes 9.1448


In [4]:
Q_mcts_avg = defaultdict(lambda: np.zeros(simulator.num_actions))
for i_trial, Q_mcts in enumerate(Q_mcts_dict[f"{best_param}"]):
    for s in range(16):
        Q_mcts_avg[s] = (1-1/(i_trial+1))*Q_mcts_avg[s] + 1/(i_trial+1)*Q_mcts[s]

Q_mcts_avg = Q_mcts_dict[f"{best_param}"][1]
for state in range(simulator.num_states):
    print(f"\n-> state = {state}")
    print(f"V[state] = {np.max(Q_mcts_avg[state]):0.4f} | {np.max(Q_vit[state]):0.4f}")
    for action in range(simulator.num_actions):
        print(f"Q[state][action] = {Q_mcts_avg[state][action]:0.4f} | {Q_vit[state][action]:0.4f}")
    print(f"best_action = {np.argmax(Q_mcts_avg[state])} | {np.argmax(Q_vit[state])}")


-> state = 0
V[state] = 0.0000 | -4.0000
Q[state][action] = 0.0000 | -5.0000
Q[state][action] = 0.0000 | -4.0000
Q[state][action] = 0.0000 | -4.0000
Q[state][action] = 0.0000 | -5.0000
best_action = 0 | 1

-> state = 1
V[state] = 0.0000 | -3.0000
Q[state][action] = 0.0000 | -5.0000
Q[state][action] = 0.0000 | -100.0000
Q[state][action] = 0.0000 | -3.0000
Q[state][action] = 0.0000 | -4.0000
best_action = 0 | 2

-> state = 2
V[state] = 0.0000 | -2.0000
Q[state][action] = 0.0000 | -4.0000
Q[state][action] = 0.0000 | -2.0000
Q[state][action] = 0.0000 | -4.0000
Q[state][action] = 0.0000 | -3.0000
best_action = 0 | 1

-> state = 3
V[state] = 0.0000 | -3.0000
Q[state][action] = 0.0000 | -3.0000
Q[state][action] = 0.0000 | -100.0000
Q[state][action] = 0.0000 | -4.0000
Q[state][action] = 0.0000 | -4.0000
best_action = 0 | 0

-> state = 4
V[state] = 0.0000 | -3.0000
Q[state][action] = 0.0000 | -4.0000
Q[state][action] = 0.0000 | -3.0000
Q[state][action] = 0.0000 | -100.0000
Q[state][action] = 0